<hr style="color:green" />
<h1 style="color:green">COSC2673 Assignment 2: Image Classification for Cancerous Cells</h1>
<h2 style="color:green">File 03: Basic PyTorch Fully Connected Neural Network model test on Main data</h2>
<hr style="color:green" />

<p>
In this file, in order to check how long training might take, do a quick train of an NN just on 200 images, or 1000 images
</p>

In [1]:
import pandas as pd
import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image

import data_basic_utility as dbutil
import graphing_utility as graphutil
import statistics_utility as statsutil
import a2_utility as a2util

randomSeed = dbutil.get_random_seed()

c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
dfImages = pd.read_csv("images_main.csv")

In [3]:
# Get The training Split and the Validation Split
dfImagesTrain = dfImages[dfImages["trainValTest"] == 0]
dfImagesVal = dfImages[dfImages["trainValTest"] == 1]

dfImagesTrain.head()

,ImageName,isCancerous,cellType,trainValTest
0,./Image_classification_data/patch_images\1.png,0,0,0
1,./Image_classification_data/patch_images\10.png,0,0,0
3,./Image_classification_data/patch_images\1000.png,1,2,0
4,./Image_classification_data/patch_images\10000...,0,1,0
5,./Image_classification_data/patch_images\10001...,0,1,0


In [4]:
# Load the images into a numpy array. This is so we can find the mean and std of the RBG values in order to Normalize the images for the Neural Network
imgsTrain = []
for imageName in dfImagesTrain["ImageName"]:
    img = cv2.imread(imageName)
    imgsTrain.append(img)

In [5]:
# Define a transform to convert images to PyTorch tensors
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Convert images to tensors and concatenate them into a single tensor
tensor_images = torch.cat([transform(img).unsqueeze(0) for img in imgsTrain])

# Calculate the mean and standard deviation of the tensor values
train_mean = tensor_images.mean(dim=(0, 2, 3))
train_std = tensor_images.std(dim=(0, 2, 3))

print("Mean:", train_mean)
print("Standard deviation:", train_std)

Mean: tensor([0.8035, 0.5909, 0.7640])
Standard deviation: tensor([0.1246, 0.1947, 0.1714])


In [6]:
class CancerBinaryDataset(Dataset):
    def __init__(self, dfImages, img_dir, transform=None, target_transform=None): 
        self.df_images = dfImages
        self.img_labels = dfImages["isCancerous"]
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)
        
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df_images.loc[idx, "ImageName"])
        image = read_image(img_path)
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
        

In [7]:
# Create a tranform operation that also normalizes the images according to the mean and standard deviations of the images
transform_normalize = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.ToTensor(), 
    transforms.Normalize(train_mean, train_std)])

# transform_normalize = transforms.Compose([transforms.Normalize(train_mean, train_std)])


In [8]:
# do a test data loader with just a few records, to see how quickly it trains locally
# Create a custom Dataset for the 
dfImagesTrain_test = dfImagesTrain.iloc[range(1000), :].reset_index()

train_testing_data = CancerBinaryDataset(dfImagesTrain_test, "./", transform=transform_normalize)

# Create data loaders
train_testing_dataloader = DataLoader(train_testing_data, batch_size=32, shuffle=True)

In [9]:
# Display image and label.
# train_features, train_labels = next(iter(train_testing_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img, cmap="gray")
# plt.show()
# print(f"Label: {label}")

# Not working, "TypeError: Invalid shape (3, 27, 27) for image data"
# But that's okay, don't think we need to display the image in this step

In [10]:
# Create a custom Dataset for the training and validation data
training_data = CancerBinaryDataset(dfImagesTrain, "./", transform=transform_normalize)
validation_data = CancerBinaryDataset(dfImagesVal, "./", transform=transform_normalize)

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = DataLoader(training_data, batch_size=32, shuffle=True, num_workers=4)

Now, create a class for the basic, Fully Connected Neural Network. For this basic NN, we will use 3 fully connected layers. The number of features in this will be 27 x 27 x 3, or 2187.

Layer 1: Input is the images, which are 27 x 27 pixels, with 3 color values (RGB). Experiment initially with 1458 nodes
Layer 2: Input is 1458 from the the previous layer, down to 729
Layer 3: Input is 729 from the the previous layer, since this is a binary classification problem, the output will be 2 classes

In this, we will use the **ReLU** Activation Function. This is the Rectified Linear Unit function, which allows the function to become non-linear

In [11]:
# Create a class for the Neural Network
class PT_NN_IsCancerous(nn.Module):

    # In the constructor, initialize the layers to use
    def __init__(self):
        super(PT_NN_IsCancerous, self).__init__()
        self.fc1 = nn.Linear(27 * 27 * 3, 1458)
        self.fc2 = nn.Linear(1458, 729)
        self.fc3 = nn.Linear(729, 2)

    # Create the forward function, which is used in training
    def forward(self, x):
        # process through each layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        # return the result
        return x


Now train the NN, just using the test training segment (this is just 50 images). This is just to see how quickly it runs in this environment.

During training, we will use the following:
- Softmax Cross Entropy Loss as our Loss function. This is a good Loss function that basically converts scores for each class into probabilities
- The Adam Optimizer, which is a version of Gradient Descent
- Initially, just 10 epochs

In [12]:
# set the Learning Rate to use
learning_rate = 0.0001
epochsToUse = 10

net = PT_NN_IsCancerous()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(epochsToUse):
    for i, data in enumerate(train_testing_dataloader, 0):
        # Get the inputs
        inputs, labels = data

        # This should convert the image tensors into vectors
        inputs = inputs.view(-1, 27 * 27 * 3)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Perform Forward and Backward propagation then optimize the weights
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

Training on 200 images seems to take 3 seconds (locally on Nelson's computer). 1000 takes about 17 seconds

So if we train locally on 10,000 images. Maybe 150 seconds, or about 3 min?